In [26]:
from bs4 import BeautifulSoup as soup

import requests
import time
import random

search_text = "java middle"

In [27]:
def wait ():
    sleep_time = {'min': 0.1, 'max': 1}
    time.sleep(sleep_time["min"]+(random.randrange(0, 10)*0.1*(sleep_time["max"]-sleep_time["min"])))

url = "https://nn.hh.ru/search/vacancy"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:91.0) Firefox/91.0",
    "Accept": "*/*",
    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "Connection": "keep-alive"
}

params = {
    "clusters": False,
    "ored_clusters": False,
    "enable_snippets": False,
    "salary": None,
    "st": "searchVacancy",
    "text": search_text,
    "page": 0
}

def get_max_page_count():
    
    response = requests.get(url, headers = headers, params = params)
    response_parsed = soup(response.text, 'html.parser')
    

    pager_buttons = response_parsed.select("div[data-qa='pager-block'] a[class='bloko-button'][data-qa='pager-page']")  
    
    max_page_number = 0
    
    for pager_button in (pager_buttons or []):
        number = int(pager_button.get_text())
        max_page_number = number if number > max_page_number else max_page_number
        
    wait()
    print("max page count: ",max_page_number)
    return max_page_number


vacancys_data = []
vacancys_on_page = False 
vacancys_page_count = get_max_page_count()

while params['page'] < vacancys_page_count:
    
    while not vacancys_on_page:
        
        response = requests.get(url, headers = headers, params = params)
        response_parsed = soup(response.text, 'html.parser')

        # ищем блоки с вакансиями и выдергиваем url
        vacancys_on_page = response_parsed.select("div[class=vacancy-serp-item]")  
        vacancys_on_page = (vacancys_on_page if len(vacancys_on_page or [])>0 else False)

        # если нашли вакансии - составляем под них структуры с данными
        if vacancys_on_page:
            for vacancy_block in vacancys_on_page:
                link_container = vacancy_block.select("a[class='bloko-link']")
                vacancys_data.append({"url": link_container[0]['href']})

        print("page ", params['page'], ", count: ", len(vacancys_on_page or []) )
        
        # симулируем
        wait()
        
    vacancys_on_page = False

    params['page'] += 1


max page count:  35
page  0 , count:  16
page  1 , count:  0
page  1 , count:  20
page  2 , count:  20
page  3 , count:  0
page  3 , count:  0
page  3 , count:  20
page  4 , count:  20
page  5 , count:  20
page  6 , count:  20
page  7 , count:  20
page  8 , count:  20
page  9 , count:  15
page  10 , count:  20
page  11 , count:  20
page  12 , count:  20
page  13 , count:  20
page  14 , count:  20
page  15 , count:  0
page  15 , count:  20
page  16 , count:  20
page  17 , count:  20
page  18 , count:  20
page  19 , count:  20
page  20 , count:  0
page  20 , count:  0
page  20 , count:  20
page  21 , count:  20
page  22 , count:  20
page  23 , count:  20
page  24 , count:  20
page  25 , count:  0
page  25 , count:  20
page  26 , count:  0
page  26 , count:  20
page  27 , count:  20
page  28 , count:  20
page  29 , count:  0
page  29 , count:  0
page  29 , count:  0
page  29 , count:  20
page  30 , count:  20
page  31 , count:  20
page  32 , count:  20
page  33 , count:  20
page  34 , cou

In [30]:
from deep_translator import GoogleTranslator
import re
import datetime

translator = GoogleTranslator(source='ru', target='en')
    
def parse_date (raw_text):
    date_text = re.search(r'\d+\s+[^\d\s]+\s+\d+', raw_text or "").group()
    date_splited = re.split("\s+", date_text)
    translation = translator.translate(date_splited[-2])
    translated_month_words = re.split("\s+", translation)
    translation = translated_month_words[-1]
    try:
        datetime_object = datetime.datetime.strptime(translation, "%B")
    except ValueError:
        return raw_text
    return datetime.date(int(date_splited[-1]), datetime_object.month, int(date_splited[0]))



In [31]:
import codecs

# разбираем страницу вакансии
counter = 1
for data in vacancys_data:
    
    response = requests.get(data['url'], headers = headers)
    response_parsed = soup(response.text, 'html.parser')    
   
    # вытаскиваем заголовок
    
    title = response_parsed.select("h1[data-qa = 'vacancy-title']")
    data["title"] = str(title[0].get_text()) if len(title)>0 else False
    
    # зарплата бля(
    
    salary_span = response_parsed.select("span[data-qa = 'bloko-header-2'][class = 'bloko-header-2 bloko-header-2_lite']")
    
    #print(counter," of ",len(vacancys_data)," : ", salary_span.get_text())
    counter += 1
    
    data["salary"] = str(salary_span[0].get_text()) if len(salary_span)>0 else False
    
    # название компании
    
    company_name_container = response_parsed.select("a[data-qa = 'vacancy-company-name'][class = 'vacancy-company-name'] > span[data-qa = 'bloko-header-2']")
    
    data["company_name"] = company_name_container[0].get_text() if len(company_name_container)>0 else False
    
    # проверенный работодатель
    
    verified_company_link = response_parsed.select("div[class = 'vacancy-company-badge'] > a[class = 'bloko-link'][href = 'https://feedback.hh.ru/article/details/id/5951']")
    data["company_verified"] = True if len(verified_company_link)>=1 else False
    
    # адрес 
    
    full_adress = response_parsed.select("span[data-qa = 'vacancy-view-raw-address']") 
    city_or_smth = response_parsed.select("p[data-qa = 'vacancy-view-location']")
    
    city_or_smth = city_or_smth[0] if len(city_or_smth)>0 else False
    full_adress = full_adress[0] if len(full_adress)>0 else False
        
    data["adress"] = ""
    
    if city_or_smth or full_adress:
        data["adress"] = (city_or_smth or full_adress).get_text()
        
    # режим на объекте
    employment_mode = response_parsed.select("p[data-qa = 'vacancy-view-employment-mode']") 
    data["employment_mode"] = employment_mode[0].get_text() if len(employment_mode)>0 else False
    
    # опыт работы 
    experience = response_parsed.select("span[data-qa = 'vacancy-experience']") 
    data["experience"] = experience[0].get_text() if len(experience)>0 else False
    
    # parttime
    parttime = response_parsed.select("p[data-qa = 'vacancy-view-parttime-options']") 
    data["parttime"] = parttime[0].get_text() if len(parttime)>0 else False

    # accept-temporary
    accept_temporary = response_parsed.select("p[data-qa = 'vacancy-view-accept-temporary']") 
    data["accept_temporary"] = accept_temporary[0].get_text() if len(accept_temporary)>0 else False

    # full_description
    description = response_parsed.select("div[class = 'vacancy-section']") 
    data["description"] = description[0].get_text() if len(description)>0 else False
  
    # дата
    date = response_parsed.select("p[class = 'vacancy-creation-time']") 
    data["creation-time"] = date[0].get_text() if len(date)>0 else False
    
    # тэги 
    tags = response_parsed.select("div[class = 'bloko-tag-list'] > div") 
    tags = tags[0] if len(tags)>0 else False
    
    tag_list = []
    if tags:
        for tag in tags.children:
            tag_list.append(tag.get_text())
    
    data["tag"] = tag_list
    
    # на скорую руку подчищаем вывод
    for key in data:
        if type(key) == type(""):
            data[key] = codecs.encode(str(data[key]),'utf8')
            data[key] = codecs.decode(data[key],'utf8')
    
    data["creation-time"] = parse_date(data["creation-time"])
  

           
    print(counter,"of",len(vacancys_data))
    
    # симулируем
    
    wait()    



2 of 691
3 of 691
4 of 691
5 of 691
6 of 691
7 of 691
8 of 691
9 of 691
10 of 691
11 of 691
12 of 691
13 of 691
14 of 691
15 of 691
16 of 691
17 of 691
18 of 691
19 of 691
20 of 691
21 of 691
22 of 691
23 of 691
24 of 691
25 of 691
26 of 691
27 of 691
28 of 691
29 of 691
30 of 691
31 of 691
32 of 691
33 of 691
34 of 691
35 of 691
36 of 691
37 of 691
38 of 691
39 of 691
40 of 691
41 of 691
42 of 691
43 of 691
44 of 691
45 of 691
46 of 691
47 of 691
48 of 691
49 of 691
50 of 691
51 of 691
52 of 691
53 of 691
54 of 691
55 of 691
56 of 691
57 of 691
58 of 691
59 of 691
60 of 691
61 of 691
62 of 691
63 of 691
64 of 691
65 of 691
66 of 691
67 of 691
68 of 691
69 of 691
70 of 691
71 of 691
72 of 691
73 of 691
74 of 691
75 of 691
76 of 691
77 of 691
78 of 691
79 of 691
80 of 691
81 of 691
82 of 691
83 of 691
84 of 691
85 of 691
86 of 691
87 of 691
88 of 691
89 of 691
90 of 691
91 of 691
92 of 691
93 of 691
94 of 691
95 of 691
96 of 691
97 of 691
98 of 691
99 of 691
100 of 691
101 of 691
102 of

AttributeError: 'NoneType' object has no attribute 'group'

In [32]:
import pandas as pd

df = pd.DataFrame(columns= list(vacancys_data[0].keys()))

ind = 0
for data in vacancys_data:     
    if (len(df.columns)==len(data)):
        df.loc[ind] = data
        ind+=1

df.to_csv(search_text+".csv", encoding="utf8" )
pd.set_option("display.max_rows",None)
df




,url,title,salary,company_name,company_verified,adress,employment_mode,experience,parttime,accept_temporary,description,creation-time,tag
0,https://nn.hh.ru/vacancy/47977394?from=vacancy...,Java Developer — Junior to Middle (Ученик),от 120 000 до 160 000 руб. на руки,ООО Цифровые привычки,True,Москва,"Полная занятость, полный день",1–3 года,False,False,Привет! Мы в Цифровых Привычках активно растем...,2021-10-06,"['Spring Framework', 'Java', 'Git', 'Hibernate..."
1,https://nn.hh.ru/vacancy/48565881?from=vacancy...,Java разработчик,от 300 000 до 525 000 руб. до вычета налогов,HR Prime,True,Москва,"Полная занятость, полный день",1–3 года,False,False,В крупную IT-компанию требуется Middle/Senior ...,2021-10-06,"['Java', 'Spring Framework', 'SQL', 'Hibernate..."
2,https://nn.hh.ru/vacancy/48572609?from=vacancy...,Java middle developer,от 200 000 до 280 000 руб. на руки,HFLabs,True,Санкт-Петербург,"Полная занятость, гибкий график",1–3 года,False,False,О продукте «Единый клиент» собирает и поддержи...,2021-10-06,"['SQL', 'PostgreSQL', 'Java', 'Git', 'Apache M..."
3,https://nn.hh.ru/vacancy/45596328?from=vacancy...,Middle Java Developer (удаленная работа),от 230 000 руб. на руки,ООО Крит,True,"Москва, 1-й Дербеневский переулок, 5","Полная занятость, удаленная работа",1–3 года,False,False,\n\n\n\n\nРаботай на крупных IT‑проектах из до...,2021-10-07,"['Java', 'Spring Framework', 'Kafka', 'Kuberne..."
4,https://nn.hh.ru/vacancy/48572764?from=vacancy...,Java middle developer,от 200 000 до 280 000 руб. на руки,HFLabs,True,Нижний Новгород,"Полная занятость, гибкий график",1–3 года,False,False,О продукте «Единый клиент» собирает и поддержи...,2021-10-06,"['SQL', 'PostgreSQL', 'Java', 'Git', 'Apache M..."
5,https://nn.hh.ru/vacancy/48097705?from=vacancy...,Java Developer (Middle/Senior),от 270 000 до 350 000 руб. на руки,EVEN Foundation,True,"Москва, Ленинский проспект, 109","Полная занятость, удаленная работа",3–6 лет,False,False,"Компания EVEN Lab входит в группу компаний, ко...",2021-10-05,"['Английский\xa0— B2 — Средне-продвинутый', 'G..."
6,https://nn.hh.ru/vacancy/48566449?from=vacancy...,Java разработчик (Middle),от 250 000 руб. до вычета налогов,ООО Медэст,True,Москва,"Полная занятость, полный день",3–6 лет,False,False,Ищем Java-разработчика на бэкенд-часть платфор...,2021-10-06,"['Git', 'Hibernate ORM', 'Spring Framework', '..."
7,https://nn.hh.ru/vacancy/48364497?from=vacancy...,Middle Java-developer,от 150 000 до 180 000 руб. на руки,Группа BLcons,True,"Владимирская, Достоевская, Санкт-Петербург, Ру...","Полная занятость, полный день",1–3 года,False,"Возможно временное оформление: договор услуг, ...",Приглашаем опытного разработчика для разработк...,2021-10-05,"['Git', 'PostgreSQL', 'Java', 'Hibernate ORM',..."
8,https://nn.hh.ru/vacancy/47198978?from=vacancy...,Java-разработчик (middle),от 200 000 до 280 000 руб. на руки,HFLabs,True,Москва,"Полная занятость, удаленная работа",1–3 года,False,False,О продукте «Единый клиент» собирает и поддержи...,2021-10-06,"['SQL', 'Java', 'Lucene', 'Spring Framework']"
9,https://nn.hh.ru/vacancy/46556960?from=vacancy...,Разработчик Java (middle) -возможна удаленка,от 160 000 руб. на руки,"айФлекс, компания",True,Москва,"Полная занятость, полный день",1–3 года,False,False,«айФлекс» – один из лидеров в области внедрени...,2021-10-07,"['Java EE', 'SOAP', 'Java', 'Git', 'EJB', 'RES..."


In [8]:
from collections import Counter

vacancy_names = df.tag # change here to change source of data/words etc

all_tags = []
for tag in vacancy_names:
    all_tags.append(' '.join(tag))
    
cloud = Counter(all_tags)
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
cloud = ''
for x in list(all_tags):
    cloud+=x+' '
wordcloud = WordCloud(width = 800, height = 800, 

                stopwords = stopwords, 
                min_font_size = 8,background_color='white'
                     ).generate(cloud)

import matplotlib.pylab as plt
plt.figure(figsize = (16, 16)) 
plt.imshow(wordcloud)
plt.savefig('vacancy_cloud.png')

In [111]:
! git add *
! git commit -m "more stable"
! git push 

The file will have its original line endings in your working directory
The file will have its original line endings in your working directory


[main 75b1e09] more stable
 4 files changed, 5315 insertions(+), 2188 deletions(-)
 rewrite .ipynb_checkpoints/main-checkpoint.ipynb (62%)


To https://github.com/russianSlave/hh_ru_parser.git
   50fb442..75b1e09  main -> main


In [12]:
import re
import datetime

raw_text = "Вакансия опубликована 24 сентября 2021 в Минске"

date_text = re.search(r'\d+\s+[^\d\s]+\s+\d+', raw_text).group()
date_splited = re.split("\s+", date_text)
print(date_splited)

from deep_translator import GoogleTranslator
translator = GoogleTranslator(source='ru', target='en')
translation = translator.translate(text = date_splited[-2])

translated_month_words = re.split("\s+", translation)
print(translated_month_words)
translation = translated_month_words[-1]


datetime_object = datetime.datetime.strptime(translation, "%B")
final_date = datetime.date(int(date_splited[-1]), datetime_object.month, int(date_splited[0]))

print (final_date) 

text = "зп 500 000 000 - 600 00 в наносекунду"

result = re.search(r'(\d+\s+)+', text)

print(result[1])



['24', 'сентября', '2021']
['september']
2021-09-24
000 


In [1]:
! pip install deep-translator


  Found existing installation: requests 2.21.0
    Uninstalling requests-2.21.0:
      Successfully uninstalled requests-2.21.0
  Found existing installation: Click 7.0
    Uninstalling Click-7.0:
      Successfully uninstalled Click-7.0


In [92]:
import translit

ModuleNotFoundError: No module named 'translit'